# CICIDS2017

Adding required libraries:

In [ ]:
!pip3 install pickle5
!pip3 install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 KB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp38-cp38-linux_x86_64.whl size=262134 sha256=04ac82f899fd3eea7eaa1f68ad78a38feaf0d515c568526f2472775c3922458a
  Stored in directory: /root/.cache/pip/wheels/25/d4/61/dbd8edd1a0d656be7b4267c85db3b61951eb60016a0154a122
Successfully built pickle5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 9.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from sklearn.ensemble import RandomForestRegressor
import pickle5 as pickle
from category_encoders import *

First of all, we determined the attack types we want to use in the dataset we have. These attack types are DoS DDoS and Port Scan.
From the dataset we created a .csv file that covers these three attack traffic and BENIGN (Benign) traffic.
Reading the data from the dataset file, combining the DoS attacks, reducing the noise, finding the NaN values ​​and assigning the column average, making the data types int64 and float64:

In [ ]:
# df=pd.read_csv('combined2.csv')
# df_value=df[' Label'].value_counts()
# df[' Label']=df[' Label'].apply({'DoS Hulk':'DoS', 'DoS GoldenEye':'DoS','DoS Slowhttptest':'DoS','DoS slowloris':'DoS' ,'BENIGN':'BENIGN' ,'DDoS':'DDoS', 'PortScan':'PortScan'}.get)
# df2=df.drop_duplicates()
# df2_value=df2[' Label'].value_counts()
# datatype=df2.dtypes
# df2['Flow Bytes/s']=df2['Flow Bytes/s'].astype('float64')
# df2[' Flow Packets/s']=df2[' Flow Packets/s'].astype('float64')
# NaN_values=df2.isnull().sum()
# df2['Flow Bytes/s'].fillna(df2['Flow Bytes/s'].mean(),inplace=True)
# print('First (row,Column) number of dataset: {} '.format(df.shape))
# print('Combining DoS dips in Dataset Label and reducing noise:\n',df2_value)
# print('Last (row,Column) number of dataset: {}'.format(df2.shape))
# #pd.options.mode.chained_assignment = None

Converting the final dataset to .csv format

df2.to_csv('dataset.csv', sep=',', encoding='utf-8', index=False)

In the next steps, we continued by reading the dataset.csv file we had just created.

> Indented block



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/IDS\ implementation/CICIDS2017
#%cd /content/drive/My\ Drive/

Mounted at /content/drive
/content/drive/My Drive/IDS implementation/CICIDS2017


## what we did with the dataset.csv file

### Reading dataset.csv, displaying dataset:



In [ ]:
dataset=pd.read_csv('dataset.csv')
dataset

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042552,53,32215,4,2,112,152,28,28,28.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1042553,53,324,2,2,84,362,42,42,42.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1042554,58030,82,2,1,31,6,31,0,15.5,21.92031,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1042555,53,1048635,6,2,192,256,32,32,32.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [ ]:
dataset[' Label'].unique()

In [ ]:
dataset.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

Application of algorithms in binary groups (DoS,BENIGN),(DDoS,BENIGN)(PortScan,BENIGN);
First, we created new dataframes by taking rows with certain Labels from the existing dataframe.

### Creation of dataframe with DoS and BENIGN:



In [ ]:
DoS_df1=dataset[dataset[' Label']=='BENIGN']
DoS_df=DoS_df1.append(dataset[dataset[' Label']=='DoS'])
DoS_df

### Dataframe of DDoS and Benign.
> Indented block





In [ ]:
DDoS_df1=dataset[dataset[' Label']=='BENIGN']
DDoS_df=DDoS_df1.append(dataset[dataset[' Label']=='DDoS'])
DDoS_df
#757096x79

### Creation of dataframe with PortScan and BENIGN

In [ ]:
PortScan_df1=dataset[dataset[' Label']=='BENIGN']
PortScan_df=PortScan_df1.append(dataset[dataset[' Label']=='PortScan'])
PortScan_df
#719893 rows × 79 columns


### Labeling the attack traffic as abnormal and tagging the BENIGN traffic as Normal and creating the dataframe:

In [ ]:
NA_df=dataset
NA_df[' Label']=NA_df[' Label'].apply({'DoS':'Anormal','BENIGN':'Normal' ,'DDoS':'Anormal', 'PortScan':'Anormal'}.get)
NA_df

DoS_df.to_csv('DoS_dataset.csv', sep=',', encoding='utf-8', index=False)

DDoS_df.to_csv('DDoS_dataset.csv', sep=',', encoding='utf-8', index=False)

PortScan_df.to_csv('PortScan_dataset.csv', sep=',', encoding='utf-8', index=False)

NA_df.to_csv('NormalAnormal_dataset.csv', sep=',', encoding='utf-8', index=False)

NaN_values1=DoS_df.isnull().sum()
NaN_values2=DDoS_df.isnull().sum()
NaN_values3=PortScan_df.isnull().sum()
NaN_values4=NA_df.isnull().sum()
NaN_values1
NaN_values2
NaN_values3
NaN_values4


## Methods written for algorithms

In [ ]:
def train_test_dataset(df1):
    pd.set_option('mode.use_inf_as_na', True)
    #pd.DataFrame(df).fillna("NaN")
    df1.dropna(inplace=True)
    X1 = df1.drop([' Label'],axis=1)
    y1 = df1.iloc[:, -1].values.reshape(-1,1)
    enc = OrdinalEncoder().fit(X1)
    X = enc.transform(X1)
    filehandler = open("enc_features.obj","wb")
    pickle.dump(enc,filehandler)
    filehandler.close()

    enc1 = OrdinalEncoder().fit(y1)
    y = enc1.transform(y1)
    filehandler = open("enc_label.obj","wb")
    pickle.dump(enc1,filehandler)
    filehandler.close()
    y=np.ravel(y)
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.7, test_size = 0.3, random_state = 0, stratify = y)
    return  X_train, X_test, y_train, y_test

In [ ]:
def RandomForest(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(random_state = 0)
    imputer = SimpleImputer(missing_values=np.nan, strategy = "mean")
    imputer = imputer.fit(X_train)
    X_train = imputer.transform(X_train)
    X_test = imputer.transform(X_test)
    rf.fit(X_train,y_train)
    filename = 'rf1.sav'
    pickle.dump(rf, open(filename, 'wb'))
    rf_score=rf.score(X_test,y_test)
    y_predict=rf.predict(X_test)
    y_true=y_test
    print('Random Forest Accuracy:'+ str(rf_score))
    precision,recall,fscore,none= precision_recall_fscore_support(y_true, y_predict, average='weighted')
    print('Random Forest precision_recall_fscore:'+(str(precision))+(str(recall))+(str(fscore)))

    cm=confusion_matrix(y_true,y_predict)
    f,ax=plt.subplots(figsize=(5,5))
    sns.heatmap(cm,annot=True,linewidth=0.5,linecolor="red",fmt=".0f",ax=ax)
    plt.xlabel("y_pred")
    plt.ylabel("y_true")
    plt.show()
    return rf_score,precision,recall,fscore,none


In [ ]:
def DecisionTree(X_train, X_test, y_train, y_test):
    dt = DecisionTreeClassifier(random_state = 0)
    imputer = SimpleImputer(missing_values=np.nan, strategy = "mean")
    imputer = imputer.fit(X_train)
    X_train = imputer.transform(X_train)
    X_test = imputer.transform(X_test)
    dt.fit(X_train, y_train)
    filename = 'dt1.sav'
    pickle.dump(dt, open(filename, 'wb'))
    score=dt.score(X_test,y_test)
    print('Decision Tree Accuracy:'+ str(score))
   # print('Decision Tree Accuracy2:'+ str(accuracy_score(y_test, y_predict)))
    y_predict=dt.predict(X_test)
    y_true=y_test
    precision,recall,fscore,none= precision_recall_fscore_support(y_true, y_predict, average='weighted')
    print('Decision Tree precision_recall_fscore:'+(str(precision))+(str(recall))+(str(fscore)))
    cm=confusion_matrix(y_true,y_predict)
    f,ax=plt.subplots(figsize=(5,5))
    sns.heatmap(cm,annot=True,linewidth=0.5,linecolor="red",fmt=".0f",ax=ax)
    plt.xlabel("y_pred")
    plt.ylabel("y_true")
    plt.show()
    return score,precision,recall,fscore,none

In [ ]:
def kNN(X_train, X_test, y_train, y_test):
    knn=KNeighborsClassifier(n_neighbors=5)
    imputer = SimpleImputer(missing_values=np.nan, strategy = "mean")
    imputer = imputer.fit(X_train)
    X_train = imputer.transform(X_train)
    X_test = imputer.transform(X_test)
    knn.fit(X_train,y_train)
    filename = 'knn.sav'
    pickle.dump(knn, open(filename, 'wb'))
    prediction=knn.predict(X_test)
    score=knn.score(X_test,y_test)
    print("5 nn score:"+ str(score))
    y_predict=knn.predict(X_test)
    y_true=y_test
    precision,recall,fscore,none= precision_recall_fscore_support(y_true, y_predict, average='weighted')
    print('5nn precision_recall_fscore:'+(str(precision))+(str(recall))+(str(fscore)))
    cm=confusion_matrix(y_true,y_predict)
    f,ax=plt.subplots(figsize=(5,5))
    sns.heatmap(cm,annot=True,linewidth=0.5,linecolor="red",fmt=".0f",ax=ax)
    plt.xlabel("y_pred")
    plt.ylabel("y_true")
    plt.show()
    return score,precision,recall,fscore,none


In [ ]:
def SVM(X_train, X_test, y_train, y_test):
    svclassifier = SVC(kernel='linear')
    imputer = SimpleImputer(missing_values=np.nan ,strategy = "mean")
    imputer = imputer.fit(X_train)
    X_train = imputer.transform(X_train)
    X_test = imputer.transform(X_test)
    svclassifier.fit(X_train, y_train)
    filename = 'svm.sav'
    pickle.dump(svclassifier, open(filename, 'wb'))
    print("SVM Classification Accuracy:"+ str(svclassifier.score(X_test,y_test)))
    y_predict = svclassifier.predict(X_test)
    y_true=y_test
    cm=confusion_matrix(y_true,y_predict)
    f,ax=plt.subplots(figsize=(5,5))
    sns.heatmap(cm,annot=True,linewidth=0.5,linecolor="red",fmt=".0f",ax=ax)
    plt.xlabel("y_pred")
    plt.ylabel("y_true")
    plt.show()

In [ ]:
def build_classifier(X_train):
    def bm():
        classifier = Sequential() # initialize neural network
        classifier.add(Dense(units = 80, kernel_initializer = 'uniform', activation = 'relu', input_dim = X_train.shape[1]))
        classifier.add(Dense(units = 25, kernel_initializer = 'uniform', activation = 'relu'))
       # classifier.add(Dense(units = 25, kernel_initializer = 'uniform', activation = 'relu'))
        classifier.add(Dense(units = 2, kernel_initializer = 'uniform', activation = 'softmax'))
        lr=.003
        adam0=Adam(lr=lr)

        classifier.compile(optimizer =adam0, loss = ' transfer type to binary.ategorical_crossentropy', metrics = ['accuracy'])
        return classifier
    return bm

In [ ]:
def ANN(X_train, X_test, y_train, y_test):
    y_ = to_categorical(y_train)
    y_t=to_categorical(y_test)
    estimator  = KerasClassifier(build_fn = build_classifier(X_train), epochs = 5)
    accuracies = cross_val_score(estimator, X = X_train, y = y_, cv = 3)
    mean = accuracies.mean()
    variance = accuracies.std()
    print("Accuracy mean: "+ str(mean))
    print("Accuracy variance: "+ str(variance))
    filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
def feature_selection(df):
    pd.set_option('mode.use_inf_as_na', True)
    #pd.DataFrame(df).fillna("NaN")
    df.dropna(inplace=True)
    feature=(df.drop([' Label'],axis=1)).columns.values
    labelencoder = LabelEncoder()
    df.iloc[:, -1] = labelencoder.fit_transform(df.iloc[:, -1])
    X = df.drop([' Label'],axis=1)
    Y = df.iloc[:, -1].values.reshape(-1,1)
    Y=np.ravel(Y)
    imputer = SimpleImputer(strategy = "mean")
    imputer = imputer.fit(X)
    X = imputer.transform(X)
    rf = RandomForestRegressor()
    rf.fit(X, Y)
    print ("Features sorted by their score:")
    print (sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), feature), reverse=True))

## Feature

In [ ]:
feature_selection(dataset)

[(0.4612, ' Bwd Packet Length Std'), (0.2906, ' ECE Flag Count'), (0.1242, ' Bwd URG Flags'), (0.037, ' Subflow Bwd Packets'), (0.0325, ' Destination Port'), (0.011, ' Min Packet Length'), (0.0079, ' Bwd Packets/s'), (0.0069, ' Flow IAT Min'), (0.0058, ' min_seg_size_forward'), (0.0039, 'Bwd IAT Total'), (0.003, ' Subflow Bwd Bytes'), (0.0027, ' Flow Packets/s'), (0.002, ' Flow IAT Std'), (0.0011, ' Fwd Packet Length Std'), (0.0008, ' PSH Flag Count'), (0.0007, 'Fwd Packets/s'), (0.0006, 'Subflow Fwd Packets'), (0.0006, ' SYN Flag Count'), (0.0006, ' Fwd IAT Min'), (0.0006, ' Bwd IAT Std'), (0.0005, ' RST Flag Count'), (0.0004, 'Fwd IAT Total'), (0.0004, ' Total Backward Packets'), (0.0004, ' Fwd URG Flags'), (0.0004, ' Fwd Packet Length Max'), (0.0003, 'Flow Bytes/s'), (0.0003, ' Fwd Packet Length Min'), (0.0003, ' Fwd IAT Mean'), (0.0003, ' Flow IAT Max'), (0.0002, 'Total Length of Fwd Packets'), (0.0002, ' Total Length of Bwd Packets'), (0.0002, ' Subflow Fwd Bytes'), (0.0002, ' Fwd Header Length'), (0.0002, ' Flow IAT Mean'), (0.0002, ' Flow Duration'), (0.0002, ' Down/Up Ratio'), (0.0002, ' CWE Flag Count'), (0.0002, ' Bwd Header Length'), (0.0001, 'Bwd Packet Length Max'), (0.0001, 'Bwd Avg Bulk Rate'), (0.0001, ' Fwd Packet Length Mean'), (0.0001, ' Fwd IAT Std'), (0.0001, ' Bwd Packet Length Min'), (0.0001, ' Bwd Packet Length Mean'), (0.0001, ' Avg Fwd Segment Size'), (0.0001, ' Average Packet Size'), (0.0, 'Init_Win_bytes_forward'), (0.0, 'Idle Mean'), (0.0, 'Fwd PSH Flags'), (0.0, 'Fwd Avg Bytes/Bulk'), (0.0, 'FIN Flag Count'), (0.0, 'Active Mean'), (0.0, ' act_data_pkt_fwd'), (0.0, ' URG Flag Count'), (0.0, ' Total Fwd Packets'), (0.0, ' Packet Length Variance'), (0.0, ' Packet Length Std'), (0.0, ' Packet Length Mean'), (0.0, ' Max Packet Length'), (0.0, ' Init_Win_bytes_backward'), (0.0, ' Idle Std'), (0.0, ' Fwd IAT Max'), (0.0, ' Fwd Header Length.1'), (0.0, ' Fwd Avg Packets/Bulk'), (0.0, ' Fwd Avg Bulk Rate'), (0.0, ' Bwd PSH Flags'), (0.0, ' Bwd IAT Min'), (0.0, ' Bwd IAT Mean'), (0.0, ' Bwd IAT Max'), (0.0, ' Bwd Avg Packets/Bulk'), (0.0, ' Bwd Avg Bytes/Bulk'), (0.0, ' Avg Bwd Segment Size'), (0.0, ' Active Std'), (0.0, ' Active Min'), (0.0, ' Active Max'), (0.0, ' ACK Flag Count')]

In [ ]:
dataset1 = dataset[[' Bwd Packet Length Std', ' ECE Flag Count',' Bwd URG Flags',' Subflow Bwd Packets',' Destination Port',' Min Packet Length',' Bwd Packets/s',' Flow IAT Min',' min_seg_size_forward','Bwd IAT Total',' Subflow Bwd Bytes',' Flow Packets/s',' Flow IAT Std', ' Fwd Packet Length Std',' PSH Flag Count','Fwd Packets/s','Subflow Fwd Packets',' SYN Flag Count',' Fwd IAT Min',' Bwd IAT Std', ' RST Flag Count','Fwd IAT Total',' Total Backward Packets',' Fwd URG Flags',' Fwd Packet Length Max','Flow Bytes/s',' Fwd Packet Length Min',' Fwd IAT Mean',' Flow IAT Max', ' Label']]

Creating Train and Test datasets for all dataframes

In [ ]:
DoSX_train, DoSX_test, DoSy_train, DoSy_test=train_test_dataset(dataset)
# DDoSX_train, DDoSX_test, DDoSy_train, DDoSy_test=train_test_dataset(DDoS_df)
# PS_X_train,PS_X_test,PS_y_train, PS_y_test=train_test_dataset(PortScan_df)
# NA_X_train, NA_X_test, NA_y_train, NA_y_test=train_test_dataset(NA_df)

### Algorithms for dataframe with DoS and BENIGN:

In [ ]:
dosrf_score,dosrf_precision,dosrf_recall,dosrf_fscore,none=RandomForest(DoSX_train, DoSX_test, DoSy_train, DoSy_test)

In [ ]:
loaded_model = pickle.load(open("rf.sav", 'rb'))
# result = loaded_model.score(DoSX_test, DoSy_test)
# print(result)
result = loaded_model.predict(DoSX_test)
list_set = set(result)
# convert the set to the list
unique_list = (list(list_set))
for x in unique_list:
    print(x)

In [ ]:
DoSX_train1, DoSX_test1, DoSy_train1, DoSy_test1 =train_test_dataset(dataset1)

In [ ]:
dosdt_score,dosdt_precision,dosdt_recall,dosdt_fscore,none=DecisionTree(DoSX_train, DoSX_test, DoSy_train, DoSy_test)

In [ ]:
dosKnn_score,dosKnn_precision,dosKnn_recall,dosKnn_fscore,none=kNN(DoSX_train, DoSX_test, DoSy_train, DoSy_test)

In [ ]:
SVM(DoSX_train, DoSX_test, DoSy_train, DoSy_test)

ANN accuracy acc:0.7636

In [ ]:
ANN(DoSX_train, DoSX_test, DoSy_train, DoSy_test)

In [ ]:
labels=np.unique(DoSy_train)
print(labels)

### Algorithms for dataframe with PortScan and BENIGN:




In [ ]:
psrf_score,psrf_precision,psrf_recall,psrf_fscore,none=RandomForest(PS_X_train,PS_X_test,PS_y_train, PS_y_test)

In [ ]:
psdt_score,psdt_precision,psdt_recall,psdt_fscore,none=DecisionTree(PS_X_train,PS_X_test,PS_y_train, PS_y_test)

In [ ]:
psKnn_score,psKnn_precision,psKnn_recall,psKnn_fscore,none=kNN(PS_X_train,PS_X_test,PS_y_train, PS_y_test)

In [ ]:
SVM(PS_X_train,PS_X_test,PS_y_train, PS_y_test)

ANN accuracy:0.8738

In [ ]:
ANN(PS_X_train,PS_X_test,PS_y_train, PS_y_test)

### Algorithms for dataframe with DDoS and BENIGN:

In [ ]:
ddosrf_score,ddosrf_precision,ddosrf_recall,ddosrf_fscore,none=RandomForest(DDoSX_train, DDoSX_test, DDoSy_train, DDoSy_test)

In [ ]:
ddosdt_score,ddosdt_precision,ddosdt_recall,ddosdt_fscore,none=DecisionTree(DDoSX_train, DDoSX_test, DDoSy_train, DDoSy_test)

In [ ]:
ddosKnn_score,ddosKnn_precision,ddosKnn_recall,ddosKnn_fscore,none=kNN(DDoSX_train, DDoSX_test, DDoSy_train, DDoSy_test)

In [ ]:
SVM(DDoSX_train, DDoSX_test, DDoSy_train, DDoSy_test)

ANN accuracy:0.8307

In [ ]:
ANN(DDoSX_train, DDoSX_test, DDoSy_train, DDoSy_test)

### Algorithms for dataframe with Normal and Abnormal:

In [ ]:
narf_score,narf_precision,narf_recall,narf_fscore,none=RandomForest(NA_X_train, NA_X_test, NA_y_train, NA_y_test)

In [ ]:
nadt_score,nadt_precision,nadt_recall,nadt_fscore,none=DecisionTree(NA_X_train, NA_X_test, NA_y_train, NA_y_test)

In [ ]:
naKnn_score,naKnn_precision,naKnn_recall,naKnn_fscore,none=kNN(NA_X_train, NA_X_test, NA_y_train, NA_y_test)

In [ ]:
SVM(NA_X_train, NA_X_test, NA_y_train, NA_y_test)

ANN accuracy:0.6034

In [ ]:
ANN(NA_X_train, NA_X_test, NA_y_train, NA_y_test)

## Accuracy table

In [ ]:
d={'Algorithms': ["Random Forest", "Decision Tree","KNN","ANN"],
   'DoS accuracy': [dosrf_score,dosdt_score,dosKnn_score,0.7636],
   'DDoS accuracy': [ddosrf_score, ddosdt_score,ddosKnn_score,0.8307],
   'Port Scan accuracy':[psrf_score,psdt_score,psKnn_score,0.8738],
   'Normal/Anormal accuracy':[narf_score,nadt_score,naKnn_score,0.6034],
  }
dataframe= pd.DataFrame(data=d)
dataframe